# Save ins del

In [1]:
import os
import torch
import sys
sys.path.append('../../lib/exlib/src')

import sys
sys.path.append('../../src')
import sop

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sop.utils.seed_all(42)

# config
exp_config = sop.ImageNetConfig()
val_config = exp_config.get_config('val_sm')
val_config['evaluation']['batch_size'] = 16
val_config

/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may hav

{'dataset': {'name': 'imagenet_s',
  'root': '/shared_data0/weiqiuy/datasets/imagenet'},
 'training': {'batch_size': 16,
  'num_epochs': 20,
  'mask_batch_size': 64,
  'optimizer': {'name': 'adamw', 'lr': 5e-06, 'weight_decay': 0.01}},
 'evaluation': {'split': 'val', 'num_data': 1, 'batch_size': 16},
 'model': {'type': 'vit',
  'base': 'google/vit-base-patch16-224',
  'sop': '/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/best',
  'num_classes': 1000}}

In [2]:
# model
backbone_model, processor, backbone_config = sop.utils.imagenet_utils.get_model(val_config['model']['type'],
                                                                 backbone_model_name=val_config['model']['base'],
                                                                 backbone_processor_name=val_config['model']['base'],
                                                                )
backbone_model = backbone_model.to(device)

# get wrapped original model
from sop.utils.imagenet_utils import WrappedModel

original_model = WrappedModel(backbone_model, output_type='logits')
original_model = original_model.to(device)
original_model.eval();

# config
from exlib.modules.sop import SOPConfig, get_chained_attr

config = SOPConfig(os.path.join(val_config['model']['sop'], 'config.json'))

# config.group_sel_scale = 0.05

config.__dict__.update(backbone_config.__dict__)
config.num_labels = len(backbone_config.id2label)

# get sop model
from sop.utils.imagenet_utils import get_model, get_wrapped_models

wrapped_backbone_model, class_weights, projection_layer = get_wrapped_models(
    backbone_model,
    config
)
wrapped_backbone_model = wrapped_backbone_model.to(device)
wrapped_backbone_model.eval();
class_weights = class_weights.to(device)
projection_layer = projection_layer.to(device)

# sop
from exlib.modules.sop import SOPImageCls4

model = SOPImageCls4(config, wrapped_backbone_model, 
                     class_weights=class_weights, 
                     projection_layer=projection_layer)
state_dict = torch.load(os.path.join(val_config['model']['sop'], 
                                     'checkpoint.pth'))
print('Loaded step', state_dict['step'])
model.load_state_dict(state_dict['model'], strict=False)
model = model.to(device)
model.eval();

projection layer is not frozen
projection layer is not frozen
Loaded step 40100


In [3]:
val_config['model']['sop']

'/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/best'

In [4]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

In [5]:
from sop.metrics import get_ins_del_perc

In [6]:
# method = 'shap_20'
method = 'bcos'

# get results example

In [7]:
# results_ins = {}
# for method in methods:
results_ins = get_ins_del_perc(val_config, original_model, backbone_model, model, processor,
                     method, debug=True)

bcos


Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

0.10202303862794221


In [8]:
results_del = get_ins_del_perc(val_config, original_model, backbone_model, model, processor,
                     method, debug=True, deletion=True)

bcos


Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/7 [00:00<?, ?it/s]

0.1019855188461131


# Results

In [22]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

In [23]:
import torch
import numpy as np

inss_dict = {}
dels_dict = {}
inss_dict_all = {}
dels_dict_all = {}

for method in methods:
    data = torch.load(f'/shared_data0/weiqiuy/sop/results/ins_del/imagenet_s/{method}.pt')
    print(method, 'ins', np.mean(data['ins']['scores_perc']), 
          'del', np.mean(data['del']['scores_perc']))
    inss_dict[method] = data['ins']['scores_perc']
    dels_dict[method] = data['del']['scores_perc']
    inss_dict_all[method] = data['ins']
    dels_dict_all[method] = data['del']

print('raw')
for method in methods:
    print(method, 'ins', np.mean(inss_dict_all[method]['scores_mean']), 'del', np.mean(dels_dict_all[method]['scores_mean']))

shap_20 ins 0.8781494490159503 del 0.4211493296650733
rise_20 ins 0.6352594335778282 del 0.7078599741135145
lime_20 ins 0.8594826424045171 del 0.4759477342731027
sop ins 0.930032776511611 del 0.10858881190715157
fullgrad ins 0.8051354604875227 del 0.43027212324771347
gradcam ins 0.8173424777666206 del 0.4155723768524423
intgrad ins 0.6605047676880771 del 0.6637644381598764
attn ins 0.758708539975525 del 0.4173078415048209
archipelago ins 0.7187009322539243 del 0.548131956821885
mfaba ins 0.7202281690683865 del 0.5466360445832881
agi ins 0.7811045738491488 del 0.5091109386202796
ampe ins 0.7234426172037536 del 0.5809504300499239
bcos ins 0.30830641975092515 del 0.3391921275119559
xdnn ins 0.25094140601082926 del 0.2101482404597633
bagnet ins 0.6264807908463602 del 0.5946418239662056
raw
shap_20 ins 0.7777502358646838 del 0.3876450827266206
rise_20 ins 0.5705529338162356 del 0.6423300426226348
lime_20 ins 0.758227471600695 del 0.43678511113994495
sop ins 0.8118536885953442 del 0.09570955

In [24]:
inss_dict_all['shap_20']['scores_curve'][:2]

[[0.07356344163417816,
  0.9061827659606934,
  0.8340227007865906,
  0.4888898730278015,
  0.5496814250946045,
  0.41891661286354065,
  0.7974211573600769,
  0.8425856828689575,
  0.8774040341377258,
  0.843177855014801],
 [0.02947884052991867,
  0.014348462224006653,
  0.2233007550239563,
  0.9688506126403809,
  0.9742243885993958,
  0.971966564655304,
  0.979910135269165,
  0.9558345675468445,
  0.9776061773300171,
  0.9888867735862732]]

In [ ]:
inss_dict_all['shap_20']['scores_curve'][:2]

In [12]:
torch.save(inss_dict, 'inss_dict.pt')
torch.save(dels_dict, 'dels_dict.pt')

### Large Hist Occl

In [32]:
methods = [
    'shap_20',
    'rise_20',
    'lime_20',
    'sop',
    'fullgrad',
    'gradcam',
    'intgrad',
    'attn',
    'archipelago',
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet',
]

import torch
import numpy as np

inss_dict = {}
dels_dict = {}
inss_dict_all = {}
dels_dict_all = {}

for method in methods:
    data = torch.load(f'/shared_data0/weiqiuy/sop/results/ins_del_large_histogram_debug/imagenet_s/{method}.pt')
    print(method, 'ins', np.mean(data['ins']['scores_perc']), 
          'del', np.mean(data['del']['scores_perc']))
    inss_dict[method] = data['ins']['scores_perc']
    dels_dict[method] = data['del']['scores_perc']
    inss_dict_all[method] = data['ins']
    dels_dict_all[method] = data['del']
    

shap_20 ins 0.9090477068151823 del 0.5242333252455312
rise_20 ins 0.6522316493111873 del 0.7755477834332745
lime_20 ins 0.7973775729575634 del 0.5548230550443558
sop ins 0.9100916063157916 del 0.1060973498770427
fullgrad ins 0.8301797727997302 del 0.4474515193911799
gradcam ins 0.7893102530945424 del 0.5403505655021642
intgrad ins 0.6787142902324547 del 0.7862346459979908
attn ins 0.745999375443722 del 0.5123231912686607
archipelago ins 0.8240276518955131 del 0.6075639270212139
mfaba ins 0.855008472434621 del 0.6098628665440533
agi ins 0.8751735294402224 del 0.5850443297345087
ampe ins 0.7378990384990791 del 0.644818024263846
bcos ins 0.5738050214930966 del 0.3803110783321334
xdnn ins 0.24453277223487846 del 0.2543259058470084
bagnet ins 0.8784463088637628 del 0.22841799392230477


In [33]:
torch.save(inss_dict, 'inss_hist_dict.pt')
torch.save(dels_dict, 'dels_hist_dict.pt')

In [34]:
for method in methods:
    print(method, 'ins', np.mean(inss_dict_all[method]['scores_mean']), 'del', np.mean(dels_dict_all[method]['scores_mean']))

shap_20 ins 0.8426726572215557 del 0.4802574971690774
rise_20 ins 0.6013469230383635 del 0.7208185009658337
lime_20 ins 0.7479649148881435 del 0.5133805638179183
sop ins 0.8438711427152157 del 0.09831391624175012
fullgrad ins 0.7721795029938221 del 0.4144583363085985
gradcam ins 0.7288615927100182 del 0.5039183814078569
intgrad ins 0.6440981850028038 del 0.7145667709410191
attn ins 0.6921353712677956 del 0.47231785394251347
archipelago ins 0.75935916043818 del 0.5744437742978334
mfaba ins 0.7847012337297201 del 0.5560737634077668
agi ins 0.8078820109367371 del 0.5360831823199987
ampe ins 0.6852656919509172 del 0.607300833798945
bcos ins 0.317127896938473 del 0.32439465972129256
xdnn ins 0.20367704692762345 del 0.21585789590608329
bagnet ins 0.7307594064623117 del 0.1913893222808838


## small interval

In [8]:
import torch
import numpy as np

inss_dict = {}
dels_dict = {}

for method in methods:
    data = torch.load(f'/shared_data0/weiqiuy/sop/results/ins_del_small/imagenet_s/{method}.pt')
    print(method, 'ins', np.mean(data['ins']['scores_perc']), 
          'del', np.mean(data['del']['scores_perc']))
    inss_dict[method] = data['ins']['scores_perc']
    dels_dict[method] = data['del']['scores_perc']
    

shap_20 ins 0.8312070952404175 del 0.3733931121679125
rise_20 ins 0.5899226568161582 del 0.6611914735527733
lime_20 ins 0.8145335845466602 del 0.42845437629875194
sop ins 0.8901901208995272 del 0.014490387549300869
fullgrad ins 0.7587626972517474 del 0.3826255156005049
gradcam ins 0.7723649433324549 del 0.36643512528295297
intgrad ins 0.6114684528454964 del 0.6166266954599398
attn ins 0.7130465605043645 del 0.3685864494690246
archipelago ins 0.6756304067388763 del 0.5012640385330499
mfaba ins 0.673658075803686 del 0.4988031737817251
agi ins 0.7345093027277142 del 0.4615287586620554
ampe ins 0.6750267081695036 del 0.5342484187978722
bcos ins 0.25739621524825496 del 0.2879965464794855
xdnn ins 0.19897524429870847 del 0.1555251473329159
bagnet ins 0.5600991198055395 del 0.4173262493706358


In [9]:
torch.save(inss_dict, 'inss_dict_small.pt')
torch.save(dels_dict, 'dels_dict_small.pt')

In [5]:
data = torch.load(f'/shared_data0/weiqiuy/sop/results/ins_del_small/imagenet_s/sop.pt')

In [18]:
data = torch.load('/shared_data0/weiqiuy/sop/results/ins_del/imagenet_s/lime_20.pt')

In [19]:
data.keys(), data['ins'].keys(), data['ins']['scores_curve_perc'][:10]

(dict_keys(['ins', 'del']),
 dict_keys(['scores_mean', 'scores_curve', 'scores_curve_perc']),
 [0.30941758659424384,
  0.5342400458198,
  0.7519785941593089,
  0.5726131650617945,
  0.982614019327433,
  0.893907126682252,
  0.8472534268134003,
  0.9473662766869041,
  0.8753398389306111,
  0.9279017636587231])

In [22]:
import numpy as np

np.mean(data['ins']['scores_curve_perc'])

0.8209578853120209

In [ ]:
for k in results_ins['scores_curve_perc']:
    print(k, np.mean(results_ins['scores_curve_perc'][k]))

In [ ]:
results_del = []
for explainer_name in methods:
    results_del[method] = get_ins_del_perc(val_config, original_model, backbone_model, model, processor,
                     explainer_name, debug=True, deletion=True)

In [ ]:
for k in results_del['scores_curve_perc']:
    print(k, np.mean(results_del['scores_curve_perc'][k]))

# Accuracy

In [9]:
def get_acc(explainer_name, suffix='', debug=False):
    method = explainer_name.split('_')[0]
    if explainer_name == 'bagnet':
        ATTR_VAL_DATA_DIR = None
    else:
        ATTR_VAL_DATA_DIR = f'/shared_data0/weiqiuy/sop/exps/imagenet_vit_1/attributions_seg/{explainer_name.replace("-s", "")}_1_pred{suffix}/val'

    explainer = sop.utils.get_explainer(original_model, backbone_model, method.split('_')[0], device)

    val_dataset, val_dataloader = sop.utils.get_dataset(val_config['dataset']['name'], 
                                          split=val_config['evaluation']['split'], 
                                          num_data=val_config['evaluation']['num_data'],
                                          batch_size=val_config['evaluation']['batch_size'],
                                                        attr_dir=ATTR_VAL_DATA_DIR,
                                          processor=processor, debug=debug)

    corrects = []
    for bi, batch in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        if debug and bi >= 3:
            break
        # if bi != len(val_dataloader) - 1:
        #     continue
        if len(batch) == 5:
            inputs, labels, segs, attrs, idxs = batch
        else:
            inputs, labels, segs, idxs = batch
            attrs = None
        inputs, labels, segs = inputs.to(device), labels.to(device), segs.to(device)

        inputs_norm = inputs
        # inputs_norm = (inputs_norm + 1) / 2
        inputs_norm = explainer.preprocess(inputs_norm)
        print('inputs_norm', inputs_norm.shape)
        # inputs_norm = (inputs_norm + 1) / 2
        outputs = explainer.model(inputs_norm)
        probs = outputs.softmax(-1)
        preds = probs.argmax(-1)
        print(preds, labels, preds == labels)
        corrects.extend((preds == labels).cpu().tolist())
    print(sum(corrects) / len(corrects))

In [11]:
get_acc('bcos', debug=True)

Using cache found in /home/runai-home/.cache/torch/hub/B-cos_B-cos-v2_main


  0%|          | 0/1001 [00:00<?, ?it/s]

Loaded 100 images and 100 classes


  0%|          | 0/100 [00:00<?, ?it/s]

inputs_norm torch.Size([1, 6, 224, 224])
tensor([0], device='cuda:0') tensor([0], device='cuda:0') tensor([True], device='cuda:0')
inputs_norm torch.Size([1, 6, 224, 224])
tensor([1], device='cuda:0') tensor([1], device='cuda:0') tensor([True], device='cuda:0')
inputs_norm torch.Size([1, 6, 224, 224])
tensor([2], device='cuda:0') tensor([2], device='cuda:0') tensor([True], device='cuda:0')
1.0
